In [118]:
#pull data from finance yahoo news using api, convert the file to json and then a list of all news for a particular stocks, for this example, tesla. 
import requests
!pip install newsapi-python
!pip install yfinance
import yfinance as yf
from newsapi import NewsApiClient
from urllib.request import urlopen, Request
!pip install bs4
from bs4 import BeautifulSoup
import json
import pandas as pd
from datetime import datetime
from nltk.sentiment.vader import SentimentIntensityAnalyzer
!pip install TextBlob
from textblob import TextBlob
#!pip install sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import matplotlib.pyplot as plt
import numpy as np

url = "https://rapidapi.p.rapidapi.com/api/yahoo/ne/news/tsla"

headers = {
    'x-rapidapi-host': "yahoo-finance15.p.rapidapi.com",
    'x-rapidapi-key': "fe6bbed556msh5d9a20764b8f0cbp1f0d22jsnc1e17707cacf"
    }

response = requests.request("GET", url, headers=headers)
json_file = response.json()
List_of_news = json_file['item']
print(List_of_news)


[{'description': 'Tesla earnings follow blowout Q3 deliveries, as production recovered from pandemic factory shutdowns.', 'guid': '3c8875c2-7895-398e-be03-fd8b5c379dc3', 'link': 'https://finance.yahoo.com/m/3c8875c2-7895-398e-be03-fd8b5c379dc3/tesla-earnings-to-spotlight.html?.tsrc=rss', 'pubDate': 'Wed, 21 Oct 2020 13:50:44 +0000', 'title': 'Tesla Earnings To Spotlight This &#39;Linchpin&#39; For Its Electric Cars'}, {'description': 'GM appeared to reaffirm its partnership agreement with electric truck maker Nikola while talks to finalize terms drag on.', 'guid': '0d7544d0-6221-3928-8ec4-57f7faa89e7b', 'link': 'https://finance.yahoo.com/m/0d7544d0-6221-3928-8ec4-57f7faa89e7b/nikola-stock-jumps-as-gm.html?.tsrc=rss', 'pubDate': 'Wed, 21 Oct 2020 13:39:45 +0000', 'title': 'Nikola Stock Jumps As GM Backs Electric Truck Deal While Talks Continue'}, {'description': 'The Zacks Analyst Blog Highlights: Tesla, General Motors, Google, Ford and Apple', 'guid': '13ecd375-46b4-3f36-87b9-f1ba1f8f2

In [52]:
#pull data from yahoo finance for tesla's stock history
url = "https://rapidapi.p.rapidapi.com/api/yahoo/hi/history/tsla/15m"

headers = {
    'x-rapidapi-host': "yahoo-finance15.p.rapidapi.com",
    'x-rapidapi-key': "fe6bbed556msh5d9a20764b8f0cbp1f0d22jsnc1e17707cacf"
    }

response = requests.request("GET", url, headers=headers)
#print(response.text)
tsla_file = response.json()
tsla_stock_price_list = tsla_file['items']
#print(tsla_stock_price_list)

{'1600695900': {'date': '21-09-2020', 'open': 452.21, 'high': 452.21, 'low': 447.32, 'close': 448.31}, '1600696800': {'date': '21-09-2020', 'open': 448.45, 'high': 450.47, 'low': 444.95, 'close': 447.11}, '1600697700': {'date': '21-09-2020', 'open': 447.12, 'high': 448.89, 'low': 430, 'close': 430.67}, '1600698600': {'date': '21-09-2020', 'open': 430.93, 'high': 432.85, 'low': 407.07, 'close': 417.94}, '1600699500': {'date': '21-09-2020', 'open': 417.97, 'high': 428.6, 'low': 417.42, 'close': 426.45}, '1600700400': {'date': '21-09-2020', 'open': 426.4, 'high': 437.2, 'low': 426.14, 'close': 431.19}, '1600701300': {'date': '21-09-2020', 'open': 431.36, 'high': 437.99, 'low': 427.7, 'close': 434.4}, '1600702200': {'date': '21-09-2020', 'open': 434.45, 'high': 436.48, 'low': 430.31, 'close': 432.52}, '1600703100': {'date': '21-09-2020', 'open': 432.53, 'high': 440.6, 'low': 432.2, 'close': 438.89}, '1600704000': {'date': '21-09-2020', 'open': 439, 'high': 444.4, 'low': 439, 'close': 441.1

In [53]:
#make a dataframe using the list tsla_stock_price_list, which is a list of dictionaries
tsla_stock_df=pd.DataFrame(tsla_stock_price_list).T.reset_index()
tsla_stock_df['date'] =pd.to_datetime(tsla_stock_df['date'])
del tsla_stock_df['index']
#tsla_stock_df=pd.DataFrame(tsla_stock_price_list)
#tsla_stock_df['date'] =pd.to_datetime(tsla_stock_df['date'], format='%d-%m-%Y')
#tsla_stock_df['close']=tsla_stock_df['close'].astype(float)

#tsla_stock_df=tsla_stock_df.groupby('date')[['close']].median().reset_index()
#tsla_stock_df['date']=tsla_stock_df['date'].dt.strftime('%Y-%m-%d')
#print(tsla_stock_df.dtypes)
print(tsla_stock_df)

          date    open    high     low   close
0   2020-09-21  452.21  452.21  447.32  448.31
1   2020-09-21  448.45  450.47  444.95  447.11
2   2020-09-21  447.12  448.89     430  430.67
3   2020-09-21  430.93  432.85  407.07  417.94
4   2020-09-21  417.97   428.6  417.42  426.45
..         ...     ...     ...     ...     ...
569 2020-10-20   421.8   422.1  420.33  421.18
570 2020-10-20  421.13  422.59   419.9  421.87
571 2020-10-21  424.08   430.5     424  426.95
572 2020-10-21  426.97  427.86  424.73   425.8
573 2020-10-21  425.56  425.56  425.56  425.56

[574 rows x 5 columns]


In [54]:
#clean data, convert date into datetime format and split date with time. 
vader = SentimentIntensityAnalyzer()
result_df=pd.DataFrame()
for each_news in List_of_news:
    #each_news['pubDate']=datetime.fromisoformat(each_news['pubDate'])
    each_news['pubDate'] = datetime.strptime(each_news['pubDate'], '%a, %d %b %Y %H:%M:%S %z')
    df = pd.DataFrame(each_news, index=[0])
    df['date'] = [d.date() for d in df['pubDate']]
    scores = df['description'].apply(vader.polarity_scores).tolist()
    df_each_news = df[['date','title']]
    scores_df = pd.DataFrame(scores)

    # Join the DataFrames of the news and the list of dicts
    parsed_and_scored_news = df_each_news.join(scores_df, rsuffix='_right')
    result_df = pd.concat([result_df,parsed_and_scored_news])
    #print(scores)
result_df = result_df.sort_values('date')
result_df['date'] =pd.to_datetime(result_df['date'], format='%Y-%m-%d')
print(result_df) 

ValueError: Empty data passed with indices specified.

In [ ]:
#merge the stock price dataframe with sentiment scores dataframe
stock_sentiment_df = pd.merge(left=tsla_stock_df, right=result_df, how='outer', left_on='date', right_on='date')
print(stock_sentiment_df)

In [ ]:
#visualization
plt.rcParams['figure.figsize'] = [10, 6]

# Group by date and calculate the mean
#mean_scores = stock_sentiment_df.groupby(['date']).mean()

# Unstack the column ticker
#mean_scores = mean_scores.unstack()

plt.plot(stock_sentiment_df['date'], stock_sentiment_df['close'])
# Plot a bar chart with pandas
plt.bar(stock_sentiment_df['date'],stock_sentiment_df['compound']*100)
plt.show()

In [ ]:
#use a different library textblob to compute sentiment scores
def getPolarity(text):
    return TextBlob(text).sentiment.polarity
result_textblob_df=pd.DataFrame()
for news in List_of_news:
    #each_news['pubDate']=datetime.fromisoformat(each_news['pubDate'])
    #print(news)
    #news['pubDate'] = datetime.strptime(news['pubDate'], '%a, %d %b %Y %H:%M:%S %z')
    df = pd.DataFrame(news, index=[0])
    df['date'] = [d.date() for d in df['pubDate']]
    scores = df['description'].apply(getPolarity).tolist()
    df_each_news = df[['date','title']]
    scores_df = pd.DataFrame(scores)

    # Join the DataFrames of the news and the list of dicts
    parsed_and_scored_news = df_each_news.join(scores_df, rsuffix='_right')
    result_textblob_df = pd.concat([result_textblob_df,parsed_and_scored_news])
result_textblob_df = result_textblob_df.sort_values('date')
print(result_textblob_df)

Try a different data set, hopefully data size will be much bigger


In [ ]:
#since yahoo finance only provides financial news for two days, 9/8/2020 and 9/9/2020
#it is insufficient to use it to create a model, now try news data from finviz

tesla_url = 'https://finviz.com/quote.ashx?t=tsla'
req = Request(url=tesla_url, headers={'user-agent': 'my-app'})
response = urlopen(req)
tsla_html = BeautifulSoup(response, 'html')
tsla_news_table = tsla_html.find(id='news-table')
tsla_news = tsla_news_table.findAll('tr')      #extract news section from the html
#print(tsla_news)

In [ ]:
#extract the timestamp and news headline 
import re
import string
tsla_news_data = []
for row in tsla_news:
    title = row.a.text
    date_list = row.td.text.split(' ')
    #date = re.sub('\D', '', date).split(' ')
    print(date_list)
    if len(date_list)==1:
        time = date_list[0]
    else:
        date = date_list[0]
        time = date_list[1]
    tsla_news_data.append([date, time, title])    
print(tsla_news_data)

In [ ]:
#now compute sentiment scores
df =pd.DataFrame(tsla_news_data,columns=['date','time','title'])
#print(df)
vader = SentimentIntensityAnalyzer()
#f = lambda title: vader.polarity_scores(title)['compound'] # only compute the 'compound' component 
df['compound'] = df['title'].apply(vader.polarity_scores)
df['date'] =pd.to_datetime(df['date'])
print(df)

In [55]:
#merge with dataframe of tesla daily stock price
tsla_df = df.merge(tsla_stock_df, how='inner', on='date')
tsla_df=tsla_df[['date','compound','close']]
tsla_df=tsla_df.groupby('date').agg({'compound':'mean','close':'mean'})
print(tsla_df)

ValueError: You are trying to merge on object and datetime64[ns] columns. If you wish to proceed you should use pd.concat

## get news data using news api
api = NewsApiClient(api_key='9ab1bcf601ca4f80b15327966b2c3229')
tsla_articles = api.get_everything(q='tesla', sources = 'bbc news, metro, financial-times, business insider, yahoo finance, reuters, bloomberg,cnbc,fortune,cbc-news',
                                   from_param='2020-09-01', to='2020-10-20',
                                   language='en'
                                   )

In [119]:
'''api = NewsApiClient(api_key='9ab1bcf601ca4f80b15327966b2c3229') 
tsla_articles = api.get_everything(everything?q='tesla', sources = 'bbc news,finviz, metro, financial-times, business insider, yahoo finance, reuters, bloomberg,cnbc,fortune,cbc-news', 
                                   from_param='2020-09-21', to='2020-10-21', 
                                   language='en' 
                                  )
print(len(tsla_articles['articles']))'''
api_key = '92ca98e398914d28a60d029d8bfc40af'
url = 'https://newsapi.org/v2/everything?q=tesla'

parameters = { 
             'q':'tesla',
             'pagesize':100,
              'apiKey':api_key,
              'language':'en',
              'from':'2020-09-21'
              }
tsla_articles = requests.get(url, params = parameters).json()
print(tsla_articles['articles'])

[{'source': {'id': None, 'name': 'Lifehacker.com'}, 'author': 'David Murphy', 'title': "How to Watch Tesla's 'Battery Day' Today", 'description': 'It’s Tesla day! Whether you think Elon Musk is a technological savior or the next Lex Luthor, there’s bound to be some interesting news at today’s much-anticipated “Battery Day” from Tesla. And you can start streaming this event around 1:30 p.m. PT today, but…', 'url': 'https://lifehacker.com/how-to-watch-teslas-battery-day-today-1845141599', 'urlToImage': 'https://i.kinja-img.com/gawker-media/image/upload/c_fill,f_auto,fl_progressive,g_center,h_675,pg_1,q_80,w_1200/hb6ycznzp49zq0wbubzs.jpg', 'publishedAt': '2020-09-22T19:00:00Z', 'content': 'Its Tesla day! Whether you think Elon Musk is a technological savior or the next Lex Luthor, theres bound to be some interesting news at todays much-anticipated Battery Day from Tesla. And you can st… [+1835 chars]'}, {'source': {'id': 'engadget', 'name': 'Engadget'}, 'author': 'Igor Bonifacic', 'title'

In [120]:
#extract news from the list tsla_articles['articles']
dic_tsla={'title':[],'date':[]}
for news in tsla_articles['articles']:
    dic_tsla['title'].append(news['title'])
    dic_tsla['date'].append(news['publishedAt'])
tsla_news = pd.DataFrame(dic_tsla,columns=['date','title']).sort_values('date').reset_index()
del tsla_news['index']

tsla_news['date'] = pd.to_datetime(tsla_news['date'])
tsla_news['date'] = [d.date() for d in tsla_news['date']]
print(tsla_news)

          date                                              title
0   2020-09-21  Musk says 'Battery Day' unveil will not reach ...
1   2020-09-21  How to Watch Tesla’s Battery Day Event—and Wha...
2   2020-09-22  Musk sees no immediate boost from 'Battery Day...
3   2020-09-22  Elon Musk: Tesla Battery Day tech won't be mas...
4   2020-09-22  Elon Musk expected to use Tesla 'Battery Day' ...
..         ...                                                ...
95  2020-10-20  Tesla analysts hope for Musk sign on 2020 deli...
96  2020-10-21  GMC reveals the Hummer EV: 1,000 HP, 350 mile ...
97  2020-10-21  Watch GM unveil the $80,000 GMC Hummer EV righ...
98  2020-10-21  Can Tesla hit 2020 delivery goal? Investors wa...
99  2020-10-21  Tesla's China factory is about to play an even...

[100 rows x 2 columns]


In [121]:
#concatenate all titles with same date using join
tsla_news['title'] = tsla_news['title'].astype(str)
tsla_news = tsla_news.groupby('date').agg({'title':','.join}).reset_index()
#further work, data might need to be cleaned
#print(tsla_news.iloc[3,0])
print(tsla_news)

          date                                              title
0   2020-09-21  Musk says 'Battery Day' unveil will not reach ...
1   2020-09-22  Musk sees no immediate boost from 'Battery Day...
2   2020-09-23  Tesla is building a cathode plant and getting ...
3   2020-09-24  Tesla seeks to build battery cells in Berlin -...
4   2020-09-27  After Math: Tesla goes budget and California h...
5   2020-09-28  Elon Musk says Tesla will probably be producin...
6   2020-09-29  Elon Musk: Tesla May Be Overvalued Today, But ...
7   2020-09-30  Tesla to roll out China-made Model 3 cars with...
8   2020-10-01  Tesla Autopilot scores low for driver engageme...
9   2020-10-02  Tesla CEO Musk suggests India entry in 2021 - ...
10  2020-10-03  Tesla will offer a bird's eye parking view, if...
11  2020-10-05  Indonesia says in early talks with Tesla on po...
12  2020-10-06  Did you think Tesla lost its ability to surpri...
13  2020-10-07  Tesla finally launches two-factor authenticati...
14  2020-1

In [122]:
def get_subjectivity(text):
    return TextBlob(text).sentiment.subjectivity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

In [123]:
#compute sentiment scores using subjectivity and polarity
vader = SentimentIntensityAnalyzer()
#f = lambda title: vader.polarity_scores(title)['compound'] # only compute the 'compound' component 
tsla_news['subjectivity']= tsla_news['title'].apply(get_subjectivity)
tsla_news['polarity']= tsla_news['title'].apply(get_subjectivity)


In [124]:
def compute_sentiment_scores(text):
    vader = SentimentIntensityAnalyzer()
    return vader.polarity_scores(text)
    
    

In [125]:
#compute sentiment scores using vader
compound=[]
neg=[]
pos=[]
neu=[]
res=0

for i in range(0, len(tsla_news['title'])):
    res = compute_sentiment_scores(tsla_news['title'][i])
    compound.append(res['compound'])
    neg.append(res['neg'])
    pos.append(res['pos'])
    neu.append(res['neu'])
    
    



In [126]:
#store this sentiment scores to tsla_news
tsla_news['compound']=compound
tsla_news['neg']=neg
tsla_news['pos']=pos
tsla_news['neu']=neu
print(tsla_news)

          date                                              title  \
0   2020-09-21  Musk says 'Battery Day' unveil will not reach ...   
1   2020-09-22  Musk sees no immediate boost from 'Battery Day...   
2   2020-09-23  Tesla is building a cathode plant and getting ...   
3   2020-09-24  Tesla seeks to build battery cells in Berlin -...   
4   2020-09-27  After Math: Tesla goes budget and California h...   
5   2020-09-28  Elon Musk says Tesla will probably be producin...   
6   2020-09-29  Elon Musk: Tesla May Be Overvalued Today, But ...   
7   2020-09-30  Tesla to roll out China-made Model 3 cars with...   
8   2020-10-01  Tesla Autopilot scores low for driver engageme...   
9   2020-10-02  Tesla CEO Musk suggests India entry in 2021 - ...   
10  2020-10-03  Tesla will offer a bird's eye parking view, if...   
11  2020-10-05  Indonesia says in early talks with Tesla on po...   
12  2020-10-06  Did you think Tesla lost its ability to surpri...   
13  2020-10-07  Tesla finally laun

In [127]:
tsla_stock_df = yf.Ticker('tsla')
tsla_stock_df = tsla_stock_df.history(period='1mo').reset_index()
tsla_stock_df = tsla_stock_df[['Date','Open','High','Low','Close','Volume']]
tsla_stock_df = tsla_stock_df.rename(columns={'Date': 'date', 'Open': 'open','High':'high','Low':'low',
                                              'Close':'close','Volume':'volume'})
print(tsla_stock_df)

         date        open        high         low       close     volume
0  2020-09-21  453.130005  455.679993  407.070007  449.390015  109476800
1  2020-09-22  429.600006  437.760010  417.600006  424.230011   79580800
2  2020-09-23  405.160004  412.149994  375.880005  380.359985   95074200
3  2020-09-24  363.799988  399.500000  351.299988  387.790009   96561100
4  2020-09-25  393.470001  408.730011  391.299988  407.339996   67208500
5  2020-09-28  424.619995  428.079987  415.549988  421.200012   49719600
6  2020-09-29  416.000000  428.500000  411.600006  419.070007   50219300
7  2020-09-30  421.320007  433.929993  420.470001  429.010010   48145600
8  2020-10-01  440.760010  448.880005  434.420013  448.160004   50741500
9  2020-10-02  421.390015  439.130005  415.000000  415.089996   71430000
10 2020-10-05  423.350006  433.640015  419.329987  425.679993   44722800
11 2020-10-06  423.790009  428.779999  406.049988  413.980011   49146300
12 2020-10-07  419.869995  429.899994  413.850006  

In [128]:
#merge with dataframe of tesla daily stock price
tsla_news['date']=pd.to_datetime(tsla_news['date'])
tsla_news_sentiment = tsla_news.merge(tsla_stock_df, how='inner', on='date')
tsla_news_sentiment=tsla_news_sentiment[['date','open','high','low','volume','subjectivity','polarity',
                                         'compound','neg','pos','neu','close']]
#tsla_news_sentiment=tsla_news_sentiment.groupby('date').agg({'compound':'mean'})
tsla_news_sentiment = tsla_news_sentiment.set_index('date')
print(tsla_news_sentiment)
#print(tsla_news_sentiment.shape)

                  open        high         low     volume  subjectivity  \
date                                                                      
2020-09-21  453.130005  455.679993  407.070007  109476800      0.000000   
2020-09-22  429.600006  437.760010  417.600006   79580800      0.350000   
2020-09-23  405.160004  412.149994  375.880005   95074200      0.399979   
2020-09-24  363.799988  399.500000  351.299988   96561100      0.213636   
2020-09-28  424.619995  428.079987  415.549988   49719600      0.000000   
2020-09-29  416.000000  428.500000  411.600006   50219300      0.300000   
2020-09-30  421.320007  433.929993  420.470001   48145600      0.000000   
2020-10-01  440.760010  448.880005  434.420013   50741500      0.150000   
2020-10-02  421.390015  439.130005  415.000000   71430000      0.166667   
2020-10-05  423.350006  433.640015  419.329987   44722800      0.529630   
2020-10-06  423.790009  428.779999  406.049988   49146300      0.000000   
2020-10-07  419.869995  4

In [129]:
#create a column called price_change, indicate increase or decrease of stock price
tsla_news_sentiment['price_change']=0
print(tsla_news_sentiment.dtypes)
#tsla_news_sentiment['price_change']=tsla_news_sentiment['price_change'].fillna(0).astype(int)
tsla_news_sentiment.iloc[0,11]=1 #since tsla stock increased from 2020-9-18 to 2020-9-21
begin_price=449.390015
for index,price in enumerate(tsla_news_sentiment['close'][1:]):
    if price> begin_price:
        tsla_news_sentiment.iloc[index+1,11]=1
    else:
        tsla_news_sentiment.iloc[index+1,11]=0
    begin_price = price
#tsla_news_sentiment.set_index('date', inplace=True)
print(tsla_news_sentiment)    
    

open            float64
high            float64
low             float64
volume            int64
subjectivity    float64
polarity        float64
compound        float64
neg             float64
pos             float64
neu             float64
close           float64
price_change      int64
dtype: object
                  open        high         low     volume  subjectivity  \
date                                                                      
2020-09-21  453.130005  455.679993  407.070007  109476800      0.000000   
2020-09-22  429.600006  437.760010  417.600006   79580800      0.350000   
2020-09-23  405.160004  412.149994  375.880005   95074200      0.399979   
2020-09-24  363.799988  399.500000  351.299988   96561100      0.213636   
2020-09-28  424.619995  428.079987  415.549988   49719600      0.000000   
2020-09-29  416.000000  428.500000  411.600006   50219300      0.300000   
2020-09-30  421.320007  433.929993  420.470001   48145600      0.000000   
2020-10-01  440.760010 

In [130]:
#feature data set
X = tsla_news_sentiment
X = np.array(X.drop(['price_change'],1))
#target data
Y = np.array(tsla_news_sentiment['price_change'])

In [131]:
#split to 80% for training and 20% for testing
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2,random_state=0)

In [132]:
#train the model
model = LinearDiscriminantAnalysis().fit(x_train, y_train)

C:\Users\zcj10\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


In [133]:
#do prediction
prediction = model.predict(x_test)
prediction

array([0, 0, 1, 1, 1], dtype=int64)

In [134]:
y_test

array([0, 0, 1, 1, 0], dtype=int64)

In [135]:
#show model performance
print(classification_report(y_test,prediction)) #performance is pretty good!

              precision    recall  f1-score   support

           0       1.00      0.67      0.80         3
           1       0.67      1.00      0.80         2

    accuracy                           0.80         5
   macro avg       0.83      0.83      0.80         5
weighted avg       0.87      0.80      0.80         5

